# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 2 2022 2:16PM,245643,10,0085959323,ISDIN Corporation,Released
1,Sep 2 2022 2:16PM,245643,10,0085959325,ISDIN Corporation,Released
2,Sep 2 2022 2:16PM,245643,10,0085959327,ISDIN Corporation,Released
3,Sep 2 2022 2:16PM,245643,10,0085959328,ISDIN Corporation,Released
4,Sep 2 2022 2:16PM,245643,10,0085959329,ISDIN Corporation,Released
5,Sep 2 2022 2:16PM,245643,10,0085959326,ISDIN Corporation,Released
6,Sep 2 2022 2:16PM,245643,10,0085959324,ISDIN Corporation,Released
7,Sep 2 2022 2:16PM,245643,10,0085959344,ISDIN Corporation,Released
8,Sep 2 2022 2:16PM,245643,10,0085959343,ISDIN Corporation,Released
9,Sep 2 2022 2:16PM,245643,10,0085959347,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
36,245643,Released,28
37,245644,Released,1
38,245645,Released,1
39,245646,Released,1
40,245652,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
245643,NaN,NaN,28.0
245644,NaN,NaN,1.0
245645,NaN,NaN,1.0
245646,NaN,NaN,1.0
245652,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
245455,5.0,1.0,0.0
245562,8.0,5.0,1.0
245563,0.0,0.0,1.0
245564,0.0,0.0,1.0
245570,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
245455,5,1,0
245562,8,5,1
245563,0,0,1
245564,0,0,1
245570,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,245455,5,1,0
1,245562,8,5,1
2,245563,0,0,1
3,245564,0,0,1
4,245570,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,245455,5,1,
1,245562,8,5,1
2,245563,,,1
3,245564,,,1
4,245570,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 2 2022 2:16PM,245643,10,ISDIN Corporation
28,Sep 2 2022 2:01PM,245652,10,Eye Pharma Inc
29,Sep 2 2022 1:43PM,245646,21,"NBTY Global, Inc."
30,Sep 2 2022 1:42PM,245645,16,Seqirus
31,Sep 2 2022 1:40PM,245644,21,"NBTY Global, Inc."
32,Sep 2 2022 1:29PM,245642,10,Emerginnova
35,Sep 2 2022 1:29PM,245641,10,ISDIN Corporation
39,Sep 2 2022 12:55PM,245639,12,Hush Hush
42,Sep 2 2022 11:47AM,245632,12,Hush Hush
43,Sep 2 2022 11:43AM,245630,20,Alumier Labs Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 2 2022 2:16PM,245643,10,ISDIN Corporation,,,28
1,Sep 2 2022 2:01PM,245652,10,Eye Pharma Inc,,,1
2,Sep 2 2022 1:43PM,245646,21,"NBTY Global, Inc.",,,1
3,Sep 2 2022 1:42PM,245645,16,Seqirus,,,1
4,Sep 2 2022 1:40PM,245644,21,"NBTY Global, Inc.",,,1
5,Sep 2 2022 1:29PM,245642,10,Emerginnova,,,3
6,Sep 2 2022 1:29PM,245641,10,ISDIN Corporation,,,4
7,Sep 2 2022 12:55PM,245639,12,Hush Hush,,,3
8,Sep 2 2022 11:47AM,245632,12,Hush Hush,,,1
9,Sep 2 2022 11:43AM,245630,20,Alumier Labs Inc.,,,6


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 2 2022 2:16PM,245643,10,ISDIN Corporation,28,,
1,Sep 2 2022 2:01PM,245652,10,Eye Pharma Inc,1,,
2,Sep 2 2022 1:43PM,245646,21,"NBTY Global, Inc.",1,,
3,Sep 2 2022 1:42PM,245645,16,Seqirus,1,,
4,Sep 2 2022 1:40PM,245644,21,"NBTY Global, Inc.",1,,
5,Sep 2 2022 1:29PM,245642,10,Emerginnova,3,,
6,Sep 2 2022 1:29PM,245641,10,ISDIN Corporation,4,,
7,Sep 2 2022 12:55PM,245639,12,Hush Hush,3,,
8,Sep 2 2022 11:47AM,245632,12,Hush Hush,1,,
9,Sep 2 2022 11:43AM,245630,20,Alumier Labs Inc.,6,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 2 2022 2:16PM,245643,10,ISDIN Corporation,28,,
1,Sep 2 2022 2:01PM,245652,10,Eye Pharma Inc,1,,
2,Sep 2 2022 1:43PM,245646,21,"NBTY Global, Inc.",1,,
3,Sep 2 2022 1:42PM,245645,16,Seqirus,1,,
4,Sep 2 2022 1:40PM,245644,21,"NBTY Global, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 2 2022 2:16PM,245643,10,ISDIN Corporation,28.0,NaN,NaN
1,Sep 2 2022 2:01PM,245652,10,Eye Pharma Inc,1.0,NaN,NaN
2,Sep 2 2022 1:43PM,245646,21,"NBTY Global, Inc.",1.0,NaN,NaN
3,Sep 2 2022 1:42PM,245645,16,Seqirus,1.0,NaN,NaN
4,Sep 2 2022 1:40PM,245644,21,"NBTY Global, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 2 2022 2:16PM,245643,10,ISDIN Corporation,28.0,0.0,0.0
1,Sep 2 2022 2:01PM,245652,10,Eye Pharma Inc,1.0,0.0,0.0
2,Sep 2 2022 1:43PM,245646,21,"NBTY Global, Inc.",1.0,0.0,0.0
3,Sep 2 2022 1:42PM,245645,16,Seqirus,1.0,0.0,0.0
4,Sep 2 2022 1:40PM,245644,21,"NBTY Global, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2456225,82.0,20.0,3.0
12,491271,4.0,0.0,0.0
15,245595,3.0,0.0,0.0
16,2456034,9.0,1.0,0.0
17,245585,1.0,0.0,0.0
18,245624,1.0,0.0,0.0
20,1473410,19.0,7.0,13.0
21,491290,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2456225,82.0,20.0,3.0
1,12,491271,4.0,0.0,0.0
2,15,245595,3.0,0.0,0.0
3,16,2456034,9.0,1.0,0.0
4,17,245585,1.0,0.0,0.0
5,18,245624,1.0,0.0,0.0
6,20,1473410,19.0,7.0,13.0
7,21,491290,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,82.0,20.0,3.0
1,12,4.0,0.0,0.0
2,15,3.0,0.0,0.0
3,16,9.0,1.0,0.0
4,17,1.0,0.0,0.0
5,18,1.0,0.0,0.0
6,20,19.0,7.0,13.0
7,21,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,82.0
1,12,Released,4.0
2,15,Released,3.0
3,16,Released,9.0
4,17,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,17,18,20,21
Status,,,,,,,,
Completed,3.0,0.0,0.0,0.0,0.0,0.0,13.0,0.0
Executing,20.0,0.0,0.0,1.0,0.0,0.0,7.0,0.0
Released,82.0,4.0,3.0,9.0,1.0,1.0,19.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,17,18,20,21
0,Completed,3.0,0.0,0.0,0.0,0.0,0.0,13.0,0.0
1,Executing,20.0,0.0,0.0,1.0,0.0,0.0,7.0,0.0
2,Released,82.0,4.0,3.0,9.0,1.0,1.0,19.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,17,18,20,21
0,Completed,3.0,0.0,0.0,0.0,0.0,0.0,13.0,0.0
1,Executing,20.0,0.0,0.0,1.0,0.0,0.0,7.0,0.0
2,Released,82.0,4.0,3.0,9.0,1.0,1.0,19.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()